# L06-1 Lab on Central Tendencies, Standard Deviation, Histograms, and Probability 

## Assignment Instructions
Rename with your name in place of Studentname and make your edits and updates here. Run each cell one at a time to make sure you understand what the code is doing.


# Lesson 6: Probability and Descriptive Statistics

6.A. Initial setup  
6.B. Loading and reviewing data  
6.1 Introduction to statistics: coding, qualitative vs. quantitative data.  
6.2. Mean, Mode, Median, Variance, Standard Deviation  
6.3. Probability  
6.4. Normal Distribution (Bell-curve)  
6.5. Confidence Intervals, Z-Score, T-Scores  
6.6. Correlation and Causation  

## Purpose
Demonstrate an example of descriptive statistics using housing data


## 6.A. Initial setup:
1. **Update working directory to point to a directory on your computer with AmesHousing.xls file**
2. The script installs required packages:
 * _readxl_ - reads our data from an Excel spreadsheet
 * _ggplot2_ - plotting
3. R doesn't have a function to calculate mode statistic. We will create **stat\_mode** function to calculate mode, which in turn uses:
 * _tabulate_ - builds a table of the counts at each combination of factor levels
 * _which.max_ - return index of the (first) maximum of a numeric (or logical) vector
 * _match_ - returns a vector of the positions of (first) matches of its first argument in its second
 * Use ?tabulate, ?match, and ?which.max for additional help
4. R's function _mad_ calculated absolute deviation around _median_, but we need deviation around _mean_ - create a function _avgAD_ to calculate

In [ ]:
#6.A. Initial setup
##### On Windows you can use the following command to map a drive to a local directory
#> subst U: C:/Users/MyDirectory

##### Adjust working directory to match your environment ######
##### This directory should contain the data for this exercise: AmesHousing.xls


In [ ]:
# List contents of working directory
dir()

In [ ]:
# R doesn't have a method to calculate mode statistic, create one
# parameters: 
#    x - vector of values 
#    na_rm = FALSE 
# output: most frequently occuring element in x
stat_mode <- function(x, na_rm = FALSE) {
  if(na_rm){
    x = x[!is.na(x)]
  }
  
  ux <- unique(x)
  return(ux[which.max(tabulate(match(x, ux)))])
}

# R's function mad() calculated absolute deviation around median, but
# we need deviation around mean - create a function to calculate
# parameters: 
#    x - vector of values 
#    na_rm = FALSE 
# output: absolute deviation around mean
avgAD <- function(x, na_rm = FALSE)
{
  if(na_rm){
    x = x[!is.na(x)]
  }
  return(sum(abs(mean(x)-x))/length(x))
}


In [ ]:
#Load libraries
library(readxl)
library(ggplot2)
library(dplyr)
library(corrgram)
library(caret)
library(tidyverse)

# 6.B. Loading and Reviewing Data
Recall our discussion about qualitative and quantitative data. R models require us to assign numeric values to character data or “code” character data so we can apply statistical techniques.  R simplifies this procedure with _**as.factor**_ function that will assign different numbers  to each distinct qualitative variable (such as Lot Shape) in our data. Much like in the example in the lesson, addition and subtraction or comparison of qualitative data is not meaningful. For example, Irregular Lot is not bigger than regular lot - they are just different from each other (or belong to different classes).

In [ ]:
# 6.B. Load and review data
#load data
AmesHousing<-readxl::read_excel("AmesHousing.xls")

#Display first 10 rows
head(AmesHousing, 10)
str(AmesHousing)

# 6.1. Introduction to statistics: coding, qualitative vs. quantitative data.
We see that our numeric data is loaded to num typed fields.

However R needs us to code character data to numbers so we can use statistical algorithms.

R simplifies this procedure with **_as.factor()_** function that will assign different numbers  to each distinct qualitative variable (such as Lot Shape) in our data. Much like in the example in the video lesson addition and subtraction or comparison of qualitative data is not meaningful. For example Irregular Lot is not bigger than regular lot - they are just different from each other or belong to different classes.



In [ ]:
# 6.1. Introduction to statistics, including: 
#    operationalization of concepts (concept to number conversion), 
#    qualitative vs. quantitative data.

#get index of character columns
i <- sapply(AmesHousing, is.character)

#set character columns to factors
AmesHousing[i] <- lapply(AmesHousing[i], as.factor)

#check data
head(AmesHousing, 10)
str(AmesHousing)

# 6.2 Central Tendencies & Measures of Dispersion
We use several mathematical measurements to determine a central point of our distribution (or central tendency), most typically: mean, median, and mode.

$mean = \frac{1}{n}\sum{(x_i)}$, where n=number of samples

$median =  \begin{cases}
               \mbox{when count values is odd: the value on the middle of ordered values}\\
               \mbox{when count values is even: average of two value on the middle of ordered values}\\
            \end{cases}$

$mode = \mbox{most frequent value}$

$StandardDeviation = \frac{1}{n}\sqrt{\sum{(x_i - mean)^2}}$

$MeanAD = \frac{1}{n}\sum{\mid{(x_i - mean)}\mid}$

$MedianAD = \frac{b}{n}\sum{\mid{(x_i - median)}\mid}$, where $b=\frac{1}{.75\_quantile\_of\_underlying\_distribution}$
b=1.4826 for normally distributed data. 

**Notes:** 
1. _central tendency functions have a parameter **na.rm** which removes NAs from calculations_, else a function will fail when it encounters NA
2. [Leys et al. 2012](https://www.academia.edu/5324493/Detecting_outliers_Do_not_use_standard_deviation_around_the_mean_use_absolute_deviation_around_the_median) advocating for using Median Absolute Deviation for outlier detection (more in the next lesson).

In [ ]:
?group_by

In [ ]:
?summarise

In [ ]:
#6.2 Central tendencies & measure of dispersion

#group AmesHousing$SalePrice by price and calculate count in each group
grouped_pricing <- AmesHousing %>%
    group_by(SalePrice) %>%
    summarise(count=n())

# Visualize grouped pricing
plot(grouped_pricing)

#calculate and output central tendencies
print(paste("mean(SalePrice) =",mean(AmesHousing$SalePrice)))
print(paste("median(SalePrice) =",median(AmesHousing$SalePrice)))
print(paste("mode(SalePrice) =",stat_mode(AmesHousing$SalePrice)))

#mode is more interesting for factors. Note that most of the lots have regular = 'Reg'shape.
print(paste("mode(Lot Shape) =",stat_mode(AmesHousing$'Lot Shape')))
#median(AmesHousing$'Lot Shape') - wouldn't work - Error: need numeric data

#standard deviation
print(paste("standard deviation (SalePrice) =", S <- sd(AmesHousing$SalePrice)))

#MAD - median absolute deviation
print(paste("median absolute deviation (SalePrice) =", MedianAD <- mad(AmesHousing$SalePrice))) #with scaling
print(paste("median absolute deviation (SalePrice, b=1) =",MedianAD <- mad(AmesHousing$SalePrice, constant=1))) #without scaling

#MAD - mean absolute deviation
print(paste("mean absolute deviation (SalePrice) =",meanAD <- avgAD(AmesHousing$SalePrice)))


# 6.C. Viewing statistics for our data

In [ ]:
?hist

In [ ]:
#6.C. Viewing statistics for our data
#get histogram information
print(">>> hist() function buckets")
print(histinfo <- hist(AmesHousing$SalePrice, plot = FALSE))

#scale density to frequency / count
scaleDensity <- max(histinfo$counts)/max(histinfo$density)

In [ ]:
# Create a ggplot object to draw a Histogram of housing sales price overlaid with density curve.
# Assigning ggplot() to an object will allow us to examine internal details.
X <- ggplot(AmesHousing, aes(x=SalePrice), environment = environment()) + 
  geom_histogram(aes(y=..count..),      # Density on y-axis
                 bins=length(histinfo$counts),
                 color="black", fill="white") +  
  eval(substitute(geom_density(aes(y = ..density..*scaleDensity), alpha=.2, fill="#FF7777"), env = list(scaleDensity=scaleDensity))) + # Overlay with density plot
  geom_vline(aes(xintercept=mean(SalePrice), color="mean"), linetype="dashed", size=1) + #vertical line for mean
  geom_vline(aes(xintercept=median(SalePrice), color="median"), linetype="dashed", size=1) + #vertical line for mean
  geom_vline(aes(xintercept=stat_mode(SalePrice), color="mode"), linetype="dashed", size=1) + #vertical line for mode
  geom_vline(aes(xintercept=mean(SalePrice)-S, color="-1 st. deviation"), linetype="solid", size=1) + #vertical line for - 1 st. deviation
  geom_vline(aes(xintercept=mean(SalePrice)+S, color="+1 st. deviation"), linetype="solid", size=1) + #vertical line for + 1 st. deviation
  geom_vline(aes(xintercept=mean(SalePrice)-meanAD, color="-1 mean abs dev"), linetype="solid", size=1) + #vertical line for - 1 st. deviation
  geom_vline(aes(xintercept=mean(SalePrice)+meanAD, color="+1 mean abs dev"), linetype="solid", size=1) + #vertical line for + 1 st. deviation
  scale_color_manual(values = c("mean"="red", "median"="green", "mode"="blue", 
                                "-1 st. deviation"="gold", "+1 st. deviation"="gold4",
                                "-1 mean abs dev"="magenta", "+1 mean abs dev"="magenta4"
  ))

#plot histogram
print(X)

In [ ]:
#examine ggplot() histogram details, notice while we used the number of buckets from hist()
#ggplot has it's own bucketing algorithm
print(">>> ggplot() function buckets")
ggplot_build(X)$data[[1]]

# 6.3. Probability

Probability is the measure of the likelihood that an event will occur on a scale from 0 to 1. Or if put in percentage terms 0% - 100%, where 0 indicates impossibility and 1 indicates certainty. 

A simple example of probability is the tossing of a fair (unbiased) coin. We have 50% probability of heads and 50% probability of tail.

Notice that on a small number of tosses we are substantially different from 50/50 heads/tails proportion. In the small sample - we may see more than twice as many tails as we see heads. Try a quarter coin toss at home and compare you results.

In [ ]:
?sample

In [ ]:
?table

In [ ]:
# coin flip experiment
set.seed(0)

coinToss <- sample(c("heads","tails"), 10, replace = TRUE)
table(coinToss)

coinToss <- sample(c("heads","tails"), 100, replace = TRUE)
table(coinToss)

coinToss <- sample(c("heads","tails"), 1000, replace = TRUE)
table(coinToss)



# 6.4. Normal Distribution (Bell-curve), Central Limit Theorem
R has several functions that help us to explore normal distribution:
* **dnorm**(x, mean = 0, sd = 1, log = FALSE) **probability density of normal distribution** or relative likelihood for a random variable **x** to have a given value 
* **pnorm**(q, mean = 0, sd = 1, lower.tail = TRUE, log.p = FALSE) **cumulative distribution** or an area under the cuve of the function _dnrom()_ 
* **qnorm**(p, mean = 0, sd = 1, lower.tail = TRUE, log.p = FALSE) **quantile of normal distribution**, 
 * **qnorm()**  is inverse ("reverses") of _pnorm()_
 * **p** is the cummulative probability parameter 
 * returns a value z* corresponding to p, where 
   * **P(x > z*) < (1-p)** for **qnorm(p, lower.tail = TRUE)**
   * and **P(x < z*) < p** for **qnorm(p, lower.tail = FALSE)**
   * i.e. probability of observing random normally distributed variable x outside of [-z\*, z\*] interval is (1-p)*2 
   * example: we use 97.5% quantile for 95% confidence level because 97.5% gives us 2.5% probability on each tail of the distribution 2.5% + 2.5% = 5% total that x will be outside of [-z\*, z\*]
* **rnorm**(n, mean = 0, sd = 1) **random numbers from normal distribution** 


In [ ]:
?xlim

In [ ]:
?stat_function

In [ ]:
?labs

In [ ]:
# Draw various normal distribution graphs
xRange = 9 #x=[-xRange, + xRange]

ggplot(data.frame(x = c(-xRange , xRange)), aes(x = x)) + 
  xlim(c(-xRange , xRange)) + 
  stat_function(fun = dnorm, args = list(mean = 0, sd = 1), aes(colour = "N(0,1)")) + 
  stat_function(fun = dnorm, args = list(mean = 0, sd = 3), aes(colour = "N(0,3)")) + 
  stat_function(fun = dnorm, args = list(mean = 2, sd = 2), aes(colour = "N(2,2)")) +
  labs(x = "\n x", y = "dnorm(x) \n", title = "Normal Distribution N(mean, sd) \n ")

In [ ]:
#draw various cummulative probability graphs
ggplot(data.frame(x = c(-xRange , xRange)), aes(x = x)) + 
  xlim(c(-xRange , xRange)) + 
  stat_function(fun = pnorm, args = list(mean = 0, sd = 1), aes(colour = "N(0,1)")) + 
  stat_function(fun = pnorm, args = list(mean = 0, sd = 3), aes(colour = "N(0,3)")) + 
  stat_function(fun = pnorm, args = list(mean = 2, sd = 2), aes(colour = "N(2,2)")) +
  labs(x = "\n x", y = "pnorm(x) \n", title = "Cummulative Distribution N(mean, sd) \n ")


In [ ]:
#Central Limit Theorem
#distribution of sample means will be approximately normal
set.seed(0)
sampleMean <- c()
for (i in 1:1000)
{
  randomSample <- sample(c(0,1), 100, replace=TRUE)
  sampleMean[i] <- mean(randomSample)
}

#plot density function of our coin experiement
#notice that it's approximately normal
ggplot(data.frame(sampleMean), aes(x = sampleMean)) + 
           geom_density(aes(y=..density..)) 

In [ ]:
#same "normality" is true for the SalesPrice in the housing data
#even though the underlying distribution is skewed
  set.seed(0)
  sampleMean <- c()
  for (i in 1:1000)
  {
    randomSample <- sample(AmesHousing$SalePrice, 100, replace=TRUE)
    sampleMean[i] <- mean(randomSample)
  }
  
  ggplot(data.frame(sampleMean), aes(x = sampleMean)) + 
    geom_density(aes(y=..density..))



# 6.5.1. Confidence Intervals

Now let's see why 95% of normally distributed samples are between +- 2 standard deviation.  
Note that qnorm(0.975)=1.95996 ~ 2*sd  

What if the population mean is unknown, like in our example of housing prices, and we want to estimate it with a range of possible values?

We can estimate using formula:
$\mu=\bar{x}\pm[MOE]$, where
* MOE = margin of error = $\frac{z^*\sigma}{\sqrt{n}}$
* $z^*=\pm qnorm(CL)$ 
* CL = confidence level

Interval $[\bar{x}-MOE, \bar{x}+MOE]$ is exact when the population distribution is normal, and approximately correct when n is large in in other cases

In [ ]:
# 6.5 Confidence Intervals, Z-Score, T-Scores

#95% of normally distributed samples with standard deviation of 1 are in the interval [-1.95996,1.95996]
  mu <- 0
  sd <- 1
  xRange = 4

  ggplot(data.frame(x = c(-xRange , xRange)), aes(x = x)) + 
    xlim(c(-xRange , xRange)) + 
    stat_function(fun = dnorm, args = list(mean = mu, sd = sd), aes(colour = "N(mu=0,sd=1)")) +
    geom_vline(aes(xintercept=qnorm(0.975), color="upper 97.5% quantile"), linetype="solid", size=1) +
    geom_vline(aes(xintercept=qnorm(0.975, lower.tail = FALSE), color="lower 97.5% quantile"), linetype="solid", size=1) 

In [ ]:
#95% confidence interval for average house price
set.seed(5)

#for somlicity assuming the entire housing dataset is complete population 
mu <- mean(AmesHousing$SalePrice)
sd <- sd(AmesHousing$SalePrice)

#plot population distribution and 95% probability interval for Population Mean (mu)
ggplot(AmesHousing, aes(x=SalePrice)) +
  scale_x_continuous(limits=c(1E5,3E5)) +
  geom_density(aes(y = ..density..), alpha=.2, fill="#FF7777") + # Overlay with density plot
  geom_vline(aes(xintercept=mu, color="mean"), linetype="dashed", size=1) +
  geom_vline(aes(xintercept=mu-qnorm(0.975)*sd/sqrt(length(SalePrice)), color="lower margin"), size=1) +
  geom_vline(aes(xintercept=mu+qnorm(0.975)*sd/sqrt(length(SalePrice)), color="upper margin"), size=1) +
  labs(title = "Population Mean (mu) with 95% Probability Interval")

In [ ]:
#mall sample population distribution and 95% Confidence Interval for the estimate of Population Mean (mu)
#note that interval is quite "wide"
sample_size <- 25
sample <- AmesHousing[sample(1:nrow(AmesHousing), sample_size),]

ggplot(sample, aes(x=SalePrice)) +
  scale_x_continuous(limits=c(1E5,3E5)) +
  geom_density(aes(y = ..density..), alpha=.2, fill="#FF7777") + # Overlay with density plot
  geom_vline(aes(xintercept=mean(SalePrice), color="mean"), linetype="dashed", size=1) +
  geom_vline(aes(xintercept=mean(SalePrice)-qnorm(0.975)*sd/sqrt(sample_size), color="lower margin"), size=1) +
  geom_vline(aes(xintercept=mean(SalePrice)+qnorm(0.975)*sd/sqrt(sample_size), color="upper margin"), size=1) +
  labs(title = paste("95% Confidence Interval For mu Estimate, sample size=",sample_size))

In [ ]:
#larger sample population distribution and 95% Confidence Interval for the estimate of Population Mean (mu)
#note that interval is "narrow"
#note that both confidence intervals include the actual population mean
sample_size <- 250
sample <- AmesHousing[sample(1:nrow(AmesHousing), sample_size),]

ggplot(sample, aes(x=SalePrice)) +
  scale_x_continuous(limits=c(1E5,3E5)) +
  geom_density(aes(y = ..density..), alpha=.2, fill="#FF7777") + # Overlay with density plot
  geom_vline(aes(xintercept=mean(SalePrice), color="mean"), linetype="dashed", size=1) +
  geom_vline(aes(xintercept=mean(SalePrice)-qnorm(0.975)*sd/sqrt(sample_size), color="lower margin"), size=1) +
  geom_vline(aes(xintercept=mean(SalePrice)+qnorm(0.975)*sd/sqrt(sample_size), color="upper margin"), size=1) +
  labs(title = paste("95% Confidence Interval For mu Estimate, sample size=",sample_size))


# 6.5.2. Z-Score, T-Scores

$Z=\frac{x-\mu}{\sigma}$

In [ ]:
?cbind

In [ ]:
?scale

In [ ]:
#note how scaling brings data to the same order of magnitude
head(cbind(AmesHousing$SalePrice,  AmesHousing$`Gr Liv Area`), 10)
head(cbind(scale(AmesHousing$SalePrice),  scale(AmesHousing$`Gr Liv Area`)), 10)


In [ ]:
#note that scaling maintains the relationship between data and "centers" data around 0
ggplot(AmesHousing,aes(x = SalePrice, y=`Gr Liv Area`)) +
    geom_point(shape=1) 
  
ggplot(AmesHousing,aes(x = scale(SalePrice), y=scale(`Gr Liv Area`))) +
    geom_point(shape=1) 

# 6.6. Correlation

Correlation – an extent of linear relationship Expressed by correlation coefficient r: 
	r  [-1, 1]
	0 – no relationship
	-1 or 1 – perfect correlation

R has a number of functions to help working with correlation  
**cor()** - returns correlation matrix  
**findCorrelation()** - identifies variables that have correlation higher than specified cutoff with other variables; note the function doesn't return variables that are highly correlated with each other;

In [ ]:
?cor

In [ ]:
# calculate correlation matrix on numeric attributes
corMatrix <- cor(na.omit(AmesHousing[, sapply(AmesHousing, is.numeric)]))

In [ ]:
# display the correlation between 2 attributes in the matrix and the correlation coefficent
(corMatrix <- cor(na.omit(AmesHousing[, c("SalePrice", "Gr Liv Area")])))
corMatrix[c("SalePrice"),c("Gr Liv Area")]

In [ ]:
?findCorrelation

In [ ]:
# find attributes that are highly correlated ( >0.75) in the matrix
highCorr <- findCorrelation(corMatrix, cutoff=0.75,  verbose = FALSE, TRUE)
print("Fields with > .75 correlation: ")
print(paste("    ", highCorr))

In [ ]:
#Zero-out lower triangle of the matrix to only show (A,B) pairs, rather than (A,B) and (B,A)
corMatrix[lower.tri(corMatrix)] <- 0

#prepare and use gather() function to unpivot our matrix
X <- as.data.frame(corMatrix)
X$var1 <- rownames(X)
Y <- gather(X, var1, correlation)
colnames(Y)[2] = "var2"

#Display highly correlated attributes. Note the variable match to findCorrelation()
filter(arrange(Y, desc(correlation)), correlation < 1 & correlation > .75)


In [ ]:
#graphical presentation of correlation matrix
corrgram(corMatrix, order=TRUE, lower.panel=NULL, upper.panel=panel.pie, cex.labels = .5)


# End of Lab

You are now ready to acquire hands-on experience with descriptive statistics and associated plots.

## Please move on to the HW-Probability-Descriptive-Statistics
